<a href="https://colab.research.google.com/github/eluo28/DetectMySetup/blob/master/using_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Detectron2 dependencies: https://detectron2.readthedocs.io/tutorials/install.html (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'



# Make sure we can import PyTorch (what Detectron2 is built with)
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 25kB/s 
     |████████████████████████████████| 4.1MB 35.8MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101
     |████████████████████████████████| 276kB 4.9MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=ff4004dce0c518fb7b0c58ebfe592e513b464ff5d14666f7294051b46c88bffe
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-b

In [ ]:
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
     |████████████████████████████████| 6.7MB 5.4MB/s 
     |████████████████████████████████| 2.2MB 10.5MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0.1-cp36-cp36m-linux_x86_64.whl size=266512 sha256=6bafc4b6d19cd28b042261d3cadb15235dae81c941fc27fec011bb6132ba662f
  Stored in directory: /root/.cache/pip/wheels/86/19/08/49b25f258ead1f861c9ab2fc41f73636f2928859adbb0e9797
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200716-cp36-none-any.whl size=42325 sha256=d04c03f2e373bbcbea861fe53669578dbbcafcf062d9f52e4c78ce188eedafc9
  Stored in directory: /root/.cache/pip/wheels/81/99/f4/42a6bef61c07b3d78dfe6d7ebff259444c4526504cf72378d7
Successfully built pycocotools fvcore
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully unin

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger() # this logs Detectron2 information such as what the model is doing when it's training

#import wandb


# import some common libraries
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import random
from google.colab.patches import cv2_imshow

from detectron2.structures import BoxMode
import json
import pprint

import logging
import os
import argparse
from collections import OrderedDict
from detectron2.engine import default_argument_parser, default_setup, launch, DefaultPredictor
from torch.nn.parallel import DistributedDataParallel


from detectron2.data import (
    MetadataCatalog,
    DatasetCatalog
)


# import some common detectron2 utilities

from detectron2.config import get_cfg # a config of "cfg" in Detectron2 is a series of instructions for building a model
from detectron2.utils.visualizer import Visualizer # a class to help visualize Detectron2 predictions on an image


In [ ]:
!gsutil -m cp -r gs://detectmysetup/train sample_data

In [ ]:
!gsutil -m cp -r gs://detectmysetup/validation sample_data

In [ ]:
# Setup the paths for the training and validation data
train_path = "./train/"
valid_path = "./sample_data/validation/"


In [ ]:
subset_classes = ["Computer keyboard","Computer monitor","Computer mouse","Lamp","Laptop","Microphone"]

In [ ]:
dataset_name="alid_labels.json"
DatasetCatalog.register(dataset_name, lambda dataset_name=dataset_name: dataset_name)

In [ ]:
MetadataCatalog.get(dataset_name).set(thing_classes=subset_classes)

Metadata(name='alid_labels.json', thing_classes=['Computer keyboard', 'Computer monitor', 'Computer mouse', 'Lamp', 'Laptop', 'Microphone'])

In [ ]:


cfg=get_cfg()


# Setup a model config (recipe for training a Detectron2 model)



 
cfg.merge_from_file("config.yaml")
cfg.MODEL.WEIGHTS = "model_final.pth"
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.8
print(cfg.DATASETS.TEST)


# Setup a default predictor from Detectron2: https://detectron2.readthedocs.io/modules/engine.html#detectron2.engine.defaults.DefaultPredictor
predictor = DefaultPredictor(cfg)

NameError: ignored

In [ ]:
DatasetCatalog._REGISTERED.pop(dataset_name)

<function __main__.<lambda>>

In [ ]:
DatasetCatalog._REGISTERED

In [ ]:


img = cv2.imread("./setup.jpg")
outputs = predictor(img)
visualizer = Visualizer(img[:, :, ::-1],
                        metadata=MetadataCatalog.get(cfg.DATASETS.TEST),
                        scale=0.7)
visualizer = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(visualizer.get_image()[:, :, ::-1])

NameError: ignored